In [1]:
import pandas as pd

In [4]:
steam_sales_path= "../data/steam_sales.csv"

In [5]:
df = pd.read_csv(steam_sales_path)
df.head()

,Game Name,Rating,#Reviews,Discount%,Price (€),Original Price (€),Release Date,Windows,Linux,MacOS,Fetched At
0,PEAK,7.0,"77,683",-38.0,4.64,7.49,"16 Jun, 2025",1,0,0,2025-08-11 00:00
1,Cyberpunk 2077,7.0,"772,574",-65.0,20.99,59.99,"9 Dec, 2020",1,0,1,2025-08-11 00:00
2,Rust,7.0,"1,038,433",-50.0,19.99,39.99,"8 Feb, 2018",1,0,1,2025-08-11 00:00
3,Microsoft Flight Simulator (2020) 40th Anniver...,5.0,"63,905",-60.0,27.99,69.99,"17 Aug, 2020",1,0,0,2025-08-11 00:00
4,Sid Meier's Civilization VII,4.0,"34,643",-30.0,48.99,69.99,"10 Feb, 2025",1,1,1,2025-08-11 00:00
